## INICIALIZO EL CLUSTER PARA EJECUTAR SPARK

In [2]:
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

AME Started!
Welcome to AME Version 3.3.9 (January 2022)

SparkSession available as spark and AME available as api

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## IMPORT LAS LIBRERIAS

In [3]:
from pyspark.sql.functions import count , col , lit, desc, when, exp , greatest , least , max , min , round

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## FUNCION GENERICA

In [4]:
def archive_date(table, date, partition_name):
    archive = spark.read.format('bigquery').option('table', table).load().filter("ARCHIVE <= '{fecha} 00:00:00'".format(fecha = date)).select(partition_name).distinct().sort(desc(partition_name)).take(1)
    return archive[0].ARCHIVE

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## LINEAL

In [5]:
lineal_score_entel = {
"cant_consultas":+0.001926,
"subrango":-0.002686,
"numero_trabajadores":-0.0002429,
"cant_socios":+0.03367,
"ind_bed_socios":+1.056,
"ind_bolcom_socios":+1.008,
"cant_vehic":+0.001573,
"avaluo_uf2":-0.0000007575,
"region_ind_rm":-0.1645,
"cantidad_infraccion":-0.00007399,
"deuda_uf_infraccion":-0.00008137,
"cantidad_multa":-0.004963,
"deuda_uf_multa":-0.0003945,
"deuda_telco_uf":+0.00002282,
"score_empresa_telefnc_tile10":-0.1415,
"beta0":+0.5197,
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## TRANSFORMACIONES

In [38]:
import sys
sys.path.append('..')
from pyspark.sql.types import FloatType, IntegerType, StringType, LongType, DateType, ArrayType , DecimalType
from pyspark.sql.functions import to_json, struct, collect_list , round, greatest, least, substring, coalesce , udf, monotonically_increasing_id, date_add, datediff, to_date, lit, when, floor, regexp_replace, from_unixtime, unix_timestamp, add_months, col, first, last, sum, min, max, avg, count, round, months_between, countDistinct, asc, desc, row_number, year, month, trim, concat, length, exp , log
from datetime import datetime
from pyspark.sql.window import Window

def rubros(df_act_princ, df_act_sii, df_cons_per , df_rubro):

	df_act_sii = df_act_sii.withColumn("row_num", row_number().over(Window.partitionBy("rut").orderBy(desc("cod_acteco") , desc("fecha_carga"), desc("fecha_inicio_acteco")))).filter((col("row_num") == 1)).drop('row_num')
	df_act_princ = df_act_princ.withColumn("row_num", row_number().over(Window.partitionBy("rut").orderBy(desc("fecha_inicio_de_actividades") , desc("cod_act")))).filter((col("row_num") == 1)).drop('row_num')

	df_cons = df_cons_per.join(df_act_princ, on ='rut' , how='left')
	df_cons = (df_cons.join(df_act_sii, on ='rut' , how='left')
            .withColumn('fecha_inicio_de_actividades', when(col('fecha_inicio_de_actividades').isNull(), col('fecha_inicio_acteco')).otherwise(col('fecha_inicio_de_actividades')))
            .withColumn('cod_act', when(col('cod_act').isNull(), col('cod_acteco')).otherwise(col('cod_act')))
            .withColumn('actividad_principal', when(col('actividad_principal').isNull(), col('desc_acteco')).otherwise(col('actividad_principal')))
        )
	df_cons = (df_cons.join(df_rubro, on = df_cons.cod_act.astype(IntegerType()) == df_rubro.codigo_de_actividad.astype(IntegerType()), how='left').filter(col('cod_act').isNotNull()).drop('codigo_de_actividad')
            .withColumn('actividad_principal', when(col('actividad_principal').isNull(), col('descripcion')).otherwise(col('actividad_principal')))
        )

	#df_cons = df_cons.withColumnRenamed('cod_act' , 'codigo_de_actividad')
	df_cons = df_cons.selectExpr('rut', 'actividad_principal as economicactivitydescription').dropDuplicates()
  
	return df_cons

def calculo_tablon_contingencia(df_cons_per, df_cons_emp, df_file_neg, df_base_entel, df_uf, df_hh, df_contactabilidad, df_pers_sem, df_sociedades, df_vehiculos, df_bancos, df_docs_trib, df_dir, df_act_princ, df_act_sii, df_rubro):

	df_cons = df_cons_per.union(df_cons_emp)
	df_rub = rubros(df_act_princ, df_act_sii, df_cons.selectExpr('rut') , df_rubro)

	df = (df_cons.join(df_file_neg, on='rut', how='left')
               .join(df_base_entel, on='rut', how='left')
               .join(df_hh, on='rut', how='left')
               .join(df_contactabilidad, on='rut', how='left')
               .join(df_pers_sem, on='rut', how='left')
               .join(df_sociedades, on='rut', how='left')
               .join(df_vehiculos, on='rut', how='left')
               .join(df_bancos, on='rut', how='left')
               .join(df_docs_trib, on='rut', how='left')
               .join(df_dir, on='rut', how='left')
               .join(df_rub, on='rut', how='left')
               .crossJoin(df_uf)
      )

	return df

def variables_por_defecto(df):

	df = df.fillna(0, subset=['deathDate','kpi_renta_02','kpi_rangtrab_01','kpi_segfac_01','predictor1_min_hh','predictor1_prom_hh','predictor1_max_hh','renta_total_hh','renta_total_lider_hh','age','interdictflag','numberofproperties','totalpropertiesvalue','countrelationshipspartner','countrelationshipssociety','observacionestributariasnumber','totalnumbercars','totalnumberheavycars','totalnumberlightcars','totalnumberothercars','addresscount','totalbank','propertiescount'])
	df = df.fillna('', subset=['maritalStatus','rutverification','direlongitud','direlatitud','redcode','gse','lastemail','flagidentityverification','gender','name','namefull','nationality','nationalitytype','categorysii','economicactivitydescription','lasttaxperiod','lasttaxperiodnumber','rubro','startsiidate','cartaxvaluation','comercialvaluationcar'])
	df = df.fillna('0.00000', subset=['gastomovilhogar','gastoteleffijohogar','gastointernet','gastopacktelco','gastobam'])
	df = df.fillna(1, subset=['clean2_hh'])
	df = df.fillna('19000101', subset=['dateofbirth'])
	df = df.fillna('X', subset=['kpi_segsii_01'])
	df = df.fillna(150000, subset=['kpi_renta_01'])
	df = df.fillna('SC', subset=['kpi_ise_01'])
	df = df.fillna('N', subset=['kpi_pquiebra_01','kpi_pension_01','kpi_eclean_01','kpi_fonasa_01'])
	df = df.fillna('SA', subset=['kpi_actcomercial_01'])
	df = df.fillna('NO', subset=['flagdeath'])
	df = df.fillna('0,X,0,150000,SC,N,N,0,N,N,SA,1', subset=['dic_rutneg_glo_00'])
    

	return df


def menores_edad_ko(df):
	df = (df.withColumn('kpi_renta_02', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('kpi_renta_02')))
			.withColumn('kpi_rangtrab_01', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('kpi_rangtrab_01')))
			.withColumn('kpi_segfac_01', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('kpi_segfac_01')))
			.withColumn('predictor1_min_hh', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('predictor1_min_hh')))
			.withColumn('predictor1_prom_hh', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('predictor1_prom_hh')))
			.withColumn('predictor1_max_hh', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('predictor1_max_hh')))
			.withColumn('renta_total_hh', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('renta_total_hh')))
			.withColumn('renta_total_lider_hh', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('renta_total_lider_hh')))
			.withColumn('age', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('age')))
			.withColumn('flagdeath', when((col('age') < 18) & (col('ind_pj') == 0) , lit('NO')).otherwise(col('flagdeath')))
			.withColumn('interdictflag', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('interdictflag')))
			.withColumn('numberofproperties', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('numberofproperties')))
			.withColumn('totalpropertiesvalue', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('totalpropertiesvalue')))
			.withColumn('countrelationshipspartner', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('countrelationshipspartner')))
			.withColumn('countrelationshipssociety', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('countrelationshipssociety')))
			.withColumn('observacionestributariasnumber', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('observacionestributariasnumber')))
			.withColumn('totalnumbercars', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('totalnumbercars')))
			.withColumn('totalnumberheavycars', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('totalnumberheavycars')))
			.withColumn('totalnumberlightcars', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('totalnumberlightcars')))
			.withColumn('totalnumberothercars', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('totalnumberothercars')))
			.withColumn('addresscount', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('addresscount')))
			.withColumn('totalbank', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('totalbank')))
			.withColumn('propertiescount', when((col('age') < 18) & (col('ind_pj') == 0) , lit(0)).otherwise(col('propertiescount')))
			.withColumn('maritalStatus', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('maritalStatus')))
			.withColumn('rutverification', when((col('ind_pj') == 1), lit('0')).otherwise(col('rutverification')))
			.withColumn('rutverification', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('rutverification')))
			.withColumn('direlongitud', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('direlongitud')))
			.withColumn('direlatitud', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('direlatitud')))
			.withColumn('redcode', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('redcode')))
			.withColumn('gse', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('gse')))
			.withColumn('lastemail', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('lastemail')))
			.withColumn('flagidentityverification', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('flagidentityverification')))
			.withColumn('gender', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('gender')))
			.withColumn('name', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('name')))
			.withColumn('namefull', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('namefull')))
			.withColumn('nationality', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('nationality')))
			.withColumn('nationalitytype', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('nationalitytype')))
			.withColumn('categorysii', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('categorysii')))
			.withColumn('economicactivitydescription', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('economicactivitydescription')))
			.withColumn('lasttaxperiod', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('lasttaxperiod')))
			.withColumn('lasttaxperiodnumber', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('lasttaxperiodnumber')))
			.withColumn('rubro', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('rubro')))
			.withColumn('startsiidate', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('startsiidate')))
			.withColumn('cartaxvaluation', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('cartaxvaluation')))
			.withColumn('comercialvaluationcar', when((col('age') < 18) & (col('ind_pj') == 0), lit('')).otherwise(col('comercialvaluationcar')))
			.withColumn('gastomovilhogar', when((col('age') < 18) & (col('ind_pj') == 0), lit('0.00000')).otherwise(col('gastomovilhogar')))
			.withColumn('gastoteleffijohogar', when((col('age') < 18) & (col('ind_pj') == 0), lit('0.00000')).otherwise(col('gastoteleffijohogar')))
			.withColumn('gastointernet', when((col('age') < 18) & (col('ind_pj') == 0), lit('0.00000')).otherwise(col('gastointernet')))
			.withColumn('gastopacktelco', when((col('age') < 18) & (col('ind_pj') == 0), lit('0.00000')).otherwise(col('gastopacktelco')))
			.withColumn('gastobam', when((col('age') < 18) & (col('ind_pj') == 0), lit('0.00000')).otherwise(col('gastobam')))
			.withColumn('clean2_hh', when((col('age') < 18) & (col('ind_pj') == 0), lit(1)).otherwise(col('clean2_hh')))
			.withColumn('dateofbirth', when((col('age') < 18) & (col('ind_pj') == 0), lit('19000101')).otherwise(col('dateofbirth')))
			.withColumn('deathDate', when((col('age') < 18) & (col('ind_pj') == 0), lit('19000101')).otherwise(col('deathDate')))
			.withColumn('kpi_segsii_01', when((col('age') < 18) & (col('ind_pj') == 0), lit('X')).otherwise(col('kpi_segsii_01')))
			.withColumn('kpi_renta_01', when((col('age') < 18) & (col('ind_pj') == 0), lit(150000)).otherwise(col('kpi_renta_01')))
			.withColumn('kpi_ise_01', when((col('age') < 18) & (col('ind_pj') == 0), lit('SC')).otherwise(col('kpi_ise_01')))
			.withColumn('kpi_pquiebra_01', when((col('age') < 18) & (col('ind_pj') == 0), lit('N')).otherwise(col('kpi_pquiebra_01')))
			.withColumn('kpi_pension_01', when((col('age') < 18) & (col('ind_pj') == 0), lit('N')).otherwise(col('kpi_pension_01')))
			.withColumn('kpi_eclean_01', when((col('age') < 18) & (col('ind_pj') == 0), lit('N')).otherwise(col('kpi_eclean_01')))
			.withColumn('kpi_fonasa_01', when((col('age') < 18) & (col('ind_pj') == 0), lit('N')).otherwise(col('kpi_fonasa_01')))
			.withColumn('kpi_actcomercial_01', when((col('age') < 18) & (col('ind_pj') == 0), lit('SA')).otherwise(col('kpi_actcomercial_01')))
			.withColumn('dic_rutneg_glo_00', when((col('age') < 18) & (col('ind_pj') == 0), lit('0,X,0,150000,SC,N,N,0,N,N,SA,1')).otherwise(col('dic_rutneg_glo_00')))
			.withColumn('age', when((col('flagdeath') == 'SI') & (col('ind_pj') == 0), lit(0)).otherwise(col('age')))
		)
	return df


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## LLAMAR TABLAS PARA UTILIZAR

In [7]:
#CONSOLIDADO_PERSONAS (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.consolidado_la_prd", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.consolidado_la_prd").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("consolidado")        

query_consolidado = '''
        SELECT  
		      rutp as rut,
              cast(coalesce(tmouf_02,0) as DECIMAL(17,2)) as dic_montototal_uf_02,
              cast(coalesce(tmot_12,0) as DECIMAL(17,2)) as dic_montototal_uf_12,
              cast(coalesce(tmot_39,0) as DECIMAL(17,2)) as dic_montototal_uf_39,
              edad_calc as age,
              regexp_replace(fnac_26, '-', '') as dateofbirth,
              coalesce(cast(regexp_replace(fec_def, '-', '') as int), 0) as deathDate,
              case when fec_def is not null then 'SI' else 'NO' end as flagdeath,
              '' as flagidentityverification,
              sexo as gender,
              ind_interd as interdictflag,
              nombres as name,
              nombre as namefull,
              avaluo_cant as numberofproperties,
              case when avaluo > 9999999999 then 9999999999 else avaluo end as totalpropertiesvalue,
              avaluo_cant as propertiescount,
              '' as lasttaxperiod,
			  '' as lasttaxperiodnumber,
			  '' as rubro,
			  '' as startsiidate,
			  '' as cartaxvaluation,
			  '' as comercialvaluationcar,
			  '' as categorysii,
			  clean2,
			  0 as ind_pj
		FROM consolidado
		WHERE archive = "2022-05-30"
'''

df_cons_per = spark.sql(query_consolidado)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#CONSOLIDADO_EMPRESA (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.consolidado_empresa_la_prd", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.consolidado_empresa_la_prd").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("consolidado_empresa")        

query_consolidado_empresa = '''
        SELECT  
            rutid as rut,
            cast(coalesce(tmouf_02,0) as DECIMAL(17,2)) as dic_montototal_uf_02,
            cast(coalesce(tmot_12,0) as DECIMAL(17,2)) as dic_montototal_uf_12,
            cast(coalesce(tmot_39,0) as DECIMAL(17,2)) as dic_montototal_uf_39,
       		null as age,
       		'0' as dateofbirth,
       		0 as deathDate,
       		'' as flagdeath,
       		'' as flagidentityverification,
       		'' as gender,
       		0 as interdictflag,
       		razon_social as name,
            razon_social as namefull,
            avaluo_cant as numberofproperties,
            case when avaluo > 9999999999 then 9999999999 else avaluo end as totalpropertiesvalue,
            avaluo_cant as propertiescount,
            '' as lasttaxperiod,
			'' as lasttaxperiodnumber,
			'' as rubro,
			'' as startsiidate,
			'' as cartaxvaluation,
			'' as comercialvaluationcar,
			'' as categorysii,
			clean2,
			1 as ind_pj
		FROM consolidado_empresa
		WHERE archive = '2022-05-30'
'''

df_cons_emp = spark.sql(query_consolidado_empresa)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#KPI (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.kpi_entel_cambrian", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.kpi_entel_cambrian").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("kpi")

query_kpi = """
        SELECT 
            DISTINCT rut,
            CONCAT_WS(',' , CAST(KPI_SEGFAC_01 AS STRING),
            CAST(KPI_SEGSII_01 AS STRING),
            CAST(KPI_RANGTRAB_01 AS STRING),
            CAST(KPI_RENTA_01 AS STRING),
            CAST(KPI_ISE_01 AS STRING),
            CAST(KPI_PQUIEBRA_01 AS STRING),
            CAST(KPI_PENSION_01 AS STRING),
            CAST(KPI_RENTA_02 AS STRING),
            CAST(KPI_FONASA_01 AS STRING),
            CAST(KPI_ECLEAN_01 AS STRING),
            CAST(KPI_ACTCOMERCIAL_01 AS STRING),
            CAST(CLEAN2 AS STRING)) as dic_rutneg_glo_00,
            CAST(kpi_segfac_01 AS INT) AS kpi_segfac_01,
	        CAST(kpi_segsii_01 AS STRING) AS kpi_segsii_01,
	        CAST(kpi_rangtrab_01 AS INT) AS kpi_rangtrab_01,
	        CAST(kpi_renta_01 AS INT) AS kpi_renta_01,
	        CAST(kpi_ise_01 AS STRING) AS kpi_ise_01,
	        CAST(kpi_pquiebra_01 AS STRING) AS kpi_pquiebra_01,
	        CAST(kpi_pension_01 AS STRING) AS kpi_pension_01,
	        CAST(kpi_fonasa_01 AS STRING) AS kpi_fonasa_01,
	        CAST(kpi_eclean_01 AS STRING) AS kpi_eclean_01,
	        --CASE WHEN CAST(kpi_pquiebra_01 AS STRING) = ' ' THEN '\" \"' ELSE CAST(kpi_pquiebra_01 AS STRING) END AS kpi_pquiebra_01,
	        --CASE WHEN CAST(kpi_pension_01 AS STRING) = ' ' THEN '\" \"' ELSE CAST(kpi_pension_01 AS STRING) END AS kpi_pension_01,
	        CAST(kpi_renta_02 AS INT) AS kpi_renta_02,
	        --CASE WHEN CAST(kpi_fonasa_01 AS STRING) = ' ' THEN '\" \"' ELSE CAST(kpi_fonasa_01 AS STRING) END AS kpi_fonasa_01,
	        --CASE WHEN CAST(kpi_eclean_01 AS STRING) = ' ' THEN '\" \"' ELSE CAST(kpi_eclean_01 AS STRING) END AS kpi_eclean_01,
	        CAST(kpi_actcomercial_01 AS STRING) AS kpi_actcomercial_01
		FROM kpi
        WHERE archive = '2022-05-30'
"""

df_file_neg = spark.sql(query_kpi)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
#BASE ENTEL (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.base_entel_cambrian", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.base_entel_cambrian").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("base_entel")

query_base_entel = """
        SELECT 
        rut,
        rut_verification as rutverification,
        dire_longitud as direlongitud,
        dire_latitud as direlatitud,
        redcode,
        gse,
        cast(cast(regexp_replace(case when gasto_movil_hogar_uf is not null and substring(gasto_movil_hogar_uf , 1 , 1) = ',' then concat('0', gasto_movil_hogar_uf) else gasto_movil_hogar_uf end , ',' , '.') as decimal(17,5)) as string) as gastomovilhogar,
        cast(cast(regexp_replace(case when gasto_telef_fijo_hogar_uf is not null and substring(gasto_telef_fijo_hogar_uf , 1 , 1) = ',' then concat('0', gasto_telef_fijo_hogar_uf) else gasto_telef_fijo_hogar_uf end , ',' , '.') as decimal(17,5)) as string) as gastoteleffijohogar,
        cast(cast(regexp_replace(case when gasto_internet_uf is not null and substring(gasto_internet_uf , 1 , 1) = ',' then concat('0', gasto_internet_uf) else gasto_internet_uf end , ',' , '.') as decimal(17,5)) as string) as gastointernet,
        cast(cast(regexp_replace(case when gasto_pack_telco_uf is not null and substring(gasto_pack_telco_uf , 1 , 1) = ',' then concat('0', gasto_pack_telco_uf) else gasto_pack_telco_uf end , ',' , '.') as decimal(17,5)) as string) as gastopacktelco,
        cast(cast(regexp_replace(case when gasto_bam_uf is not null and substring(gasto_bam_uf , 1 , 1) = ',' then concat('0', gasto_bam_uf) else gasto_bam_uf end , ',' , '.') as decimal(17,5)) as string) as gastobam
		FROM base_entel
		WHERE archive = "2022-05-30"
"""

df_base_entel = spark.sql(query_base_entel)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#UF (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.monedas_norm_dev_dummy", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.monedas_norm_dev_dummy").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("uf")

query_uf = """
        SELECT 
        		cast(valor_uf as DECIMAL(17,2)) as valor_uf
		FROM uf a
		WHERE a.fecha in (select max(f.fecha) as fecha from uf f)
"""

df_uf = spark.sql(query_uf)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#RUTERO (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.rutero_entel_cambrian", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.rutero_entel_cambrian").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("rutero")

query_rutero = """
  		SELECT DISTINCT rut 
  		FROM rutero
"""

df_rutero = spark.sql(query_rutero)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
#HOUSEHOLD (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.hh_principal_hist_la_prd", '2022-05-24' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.hh_principal_hist_la_prd").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("household")

query_household = """
	     SELECT 
	        rut,
	        predictor1_min_hh_principal as predictor1_min_hh,
	        predictor1_hh_principal as predictor1_prom_hh,
	        predictor1_max_hh_principal as predictor1_max_hh,
	        renta_total_hh_principal as renta_total_hh,
	        renta_total_lider_hh_principal as renta_total_lider_hh,
	        porc_personas_clean2_hh_principal as clean2_hh
	    FROM household
	    WHERE archive = '2022-05-24'
"""

df_hh = spark.sql(query_household)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#CONSOLIDADO CONTABILIDAD (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.consolidado_contabilidad_cambrian", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.consolidado_contabilidad_cambrian").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("consolidado_contabilidad")

query_consolidado_contabilidad = """
        SELECT  r.rut,
		        cant_emails as lastemail
		FROM consolidado_contabilidad r
		WHERE cant_emails = 1
"""

df_contactabilidad = spark.sql(query_consolidado_contabilidad)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
#PERSONAS SEMANAL (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.personas_semanal_cambrian", '2022-06-07' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.personas_semanal_cambrian").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("persona_semanal")

query_persona_semanal = """
        SELECT  rutp as rut,
        		case when ncion = 'CHILENA' then 'CHILENO' when ncion is null then '' else 'EXTRANJERO' end as nationality,
        		coalesce(ecivi, '') as maritalStatus,
        		tinac as nationalitytype
		FROM persona_semanal
		WHERE archive = "2022-06-07"
"""

df_pers_sem = spark.sql(query_persona_semanal)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
#SCORE (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.score_entel_cambrian", '2022-05-31' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.score_entel_cambrian").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("score")

query_score = """
  		SELECT rut , score as personsRate
		FROM score
		WHERE archive= "2022-05-31"
"""

df_score = spark.sql(query_score)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
#SOCIOS Y SOCIEDADES (OK)

#dos atributos hay que desencryptar la info del rut de socios en el ws pdz

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.empresas_mensual_la_prd", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.empresas_mensual_la_prd").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("s_s")

query_s_s = """
		SELECT  rutid as rut,
		        vehiculos_cantidad as countrelationshipspartner,
		        vehiculos_avaluo_total as countrelationshipssociety
		FROM s_s 
"""

df_sociedades = spark.sql(query_s_s)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
#VEHICULOS (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.tbtas_vehiculos_la_prd", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.tbtas_vehiculos_la_prd").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("vehiculos")

query_vehiculos = """
        SELECT  rutid as rut,
		        sum(case when clasificacion = 'LIVIANO' then 1 else 0 end) as totalnumberlightcars,
		        sum(case when clasificacion = 'PESADO' then 1 else 0 end) as totalnumberheavycars,
		        sum(case when clasificacion != 'LIVIANO' and clasificacion != 'PESADO' then 1 else 0 end) as totalnumberothercars,
		        count(*) as totalnumbercars
		FROM vehiculos
		where archive = "2022-05-30"
		GROUP BY rut
"""

df_vehiculos = spark.sql(query_vehiculos)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
#CUENTAS BANCARIAS (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.personas_mensual_la_prd", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.personas_mensual_la_prd").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("cuentas_bancarias")

query_cuentas_bancaria = """
        SELECT  ps.rutp as rut,
        		count(distinct soc.cban) as totalbank
		FROM cuentas_bancarias ps
		LATERAL VIEW explode(ps.cuentas) adTable as soc
		WHERE archive = '2022-05-30'
		GROUP BY ps.rutp
		UNION ALL
		SELECT  em.rutp as rut,
		        count(distinct soc.cban) as totalbank
		FROM cuentas_bancarias em
		LATERAL VIEW explode(em.cuentas) adTable as soc
		WHERE archive = '2022-05-30'
		GROUP BY em.rutp
"""

df_bancos = spark.sql(query_cuentas_bancaria)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
#DOCUMENTOS TRIBUTARIOS (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.empresas_mensual_la_prd", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.empresas_mensual_la_prd").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("documentos_tributarios")

query_documentos_tributarios = """
        SELECT  ps.rutid as rut,
        		count(*) as observacionestributariasnumber
		FROM documentos_tributarios ps
		LATERAL VIEW explode(ps.documentos_tributarios) adTable as soc
		WHERE archive = '2022-05-30'
		GROUP BY ps.rutid
		UNION ALL
		SELECT  em.rutid as rut,
		        count(*) as observacionestributariasnumber
		FROM documentos_tributarios em
		LATERAL VIEW explode(em.documentos_tributarios) adTable as soc
		WHERE archive = '2022-05-30'
		GROUP BY em.rutid
"""

df_docs_trib = spark.sql(query_documentos_tributarios)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
#DIRECCIONES (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.personas_mensual_la_prd", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.personas_mensual_la_prd").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("direcciones")

query_direcciones = """
        SELECT  ps.rutp as rut,
        		count(*) as addresscount
		FROM direcciones ps
		LATERAL VIEW explode(ps.directorio_direccion) adTable as soc
		WHERE archive = '2022-05-30'
		GROUP BY ps.rutp
		UNION ALL
		SELECT  em.rutp as rut,
		        count(*) as addresscount
		FROM direcciones em
		LATERAL VIEW explode(em.directorio_direccion) adTable as soc
		WHERE archive = '2022-05-30'
		GROUP BY em.rutp
"""

df_dir = spark.sql(query_direcciones)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
#ACTIVIDAD PRINCIPAL (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.empresas_mensual_la_prd", '2022-04-26' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.empresas_mensual_la_prd").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("actividad_principal")

query_actividad_principal = """
        SELECT  em.rutid as rut,
				cod_act,
				actividad_principal,
				fecha_inici_contrib as fecha_inicio_de_actividades,
				fecha_const as fecha_de_constitucion,
				fecha_term_giro_contrib as fecha_termino_de_giro,
				fecha_ultimo_documento as fecha_ultimo_timbraje_de_documentos
		FROM actividad_principal em
		WHERE archive = '2022-04-26'
		and cod_act is not null
"""

df_act_princ = spark.sql(query_actividad_principal)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
#ACTIVIDAD SII (OK)


archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.empresas_mensual_la_prd", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.empresas_mensual_la_prd").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("actividad_sii")

query_actividad_sii = """
        SELECT  em.rutid as rut
		        , gir.cod_acteco
		        , gir.desc_acteco
				, gir.fecha_inicio_acteco as fecha_inicio_acteco
				, gir.fecha_carga as fecha_carga
		FROM actividad_sii em
		LATERAL VIEW explode(em.actividades_economicas) adTable as gir
		WHERE archive = '2022-05-30'
		and gir.cod_acteco != '0-9999'
		UNION ALL
		SELECT  em.rutid as rut
		        , gir.cod_acteco
		        , gir.desc_acteco
				, gir.fecha_inicio_acteco as fecha_inicio_acteco
				, gir.fecha_carga as fecha_carga
		FROM actividad_sii em
		LATERAL VIEW explode(em.actividades_economicas) adTable as gir
		WHERE archive = '2022-05-30'
		and gir.cod_acteco != '0-9999' 
"""

df_act_sii = spark.sql(query_actividad_sii)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
#RUBRO (OK)

archive_consolidado = archive_date("dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.rubros_cambrian", '2022-05-30' , 'ARCHIVE')
consolidado_data = spark.read.format("bigquery").option("table", "dfa-dna-ws0011-la-prd-f758.dfa_dna_ws0011_la_prd_sandbox.rubros_cambrian").load().filter("ARCHIVE = '{fecha}'".format(fecha = archive_consolidado))
consolidado_data.createOrReplaceTempView("rubro")

query_rubro = """
        SELECT cod as codigo_de_actividad
	    		, rubro
	    		, descripcion
	    		, rubro_especifico sub_rubro
	    FROM rubro
"""

df_rubro = spark.sql(query_rubro)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
#REVISAR TABLAS CARGADAS
df_rubro.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+--------------------+--------------------+
|codigo_de_actividad|               rubro|         descripcion|           sub_rubro|
+-------------------+--------------------+--------------------+--------------------+
|              11101|AGRICULTURA GANAD...|    CULTIVO DE TRIGO|CULTIVO DE PLANTA...|
|              11102|AGRICULTURA GANAD...|     CULTIVO DE MAIZ|CULTIVO DE PLANTA...|
|              11103|AGRICULTURA GANAD...|    CULTIVO DE AVENA|CULTIVO DE PLANTA...|
|              11104|AGRICULTURA GANAD...|   CULTIVO DE CEBADA|CULTIVO DE PLANTA...|
|              11105|AGRICULTURA GANAD...|CULTIVO DE OTROS ...|CULTIVO DE PLANTA...|
|              11106|AGRICULTURA GANAD...|  CULTIVO DE POROTOS|CULTIVO DE PLANTA...|
|              11107|AGRICULTURA GANAD...|   CULTIVO DE LUPINO|CULTIVO DE PLANTA...|
|              11108|AGRICULTURA GANAD...|CULTIVO DE OTRAS ...|CULTIVO DE PLANTA...|
|              11109|AGRICULTURA GANAD...|CULTIVO DE SEMILL...|CU

## INFORMACION CONFIGURE

In [26]:
# -- coding: UTF-8 --

# PROD
# 0: debug
# 1: production
# 2: new columns
ind_prod = 0

# Bases genericas
cl_efx_att = 'cl_efx_att'
cl_efx_pre_att = 'cl_efx_pre_att'
cl_efx_prod = 'cl_efx_prod'
cl1_g_enrichhh = 'cl1_g_enrichhh_db'
cl_efx_cli_dev = 'cl_efx_cli_dev'

# Formato
format0 = 'yyyyMM'
format1 = 'yyyy-MM'
formatoHora = '%H:%M:%S'
formatoFechaEstandar = '%Y%m%d'
formatoFechaEstandar1 = '%Y-%m-%d'
regexFecha = '^([0-9]{4}[-/]?((0[13-9]|1[012])[-/]?(0[1-9]|[12][0-9]|30)|(0[13578]|1[02])[-/]?31|02[-/]?(0[1-9]|1[0-9]|2[0-8]))|([0-9]{2}(([2468][048]|[02468][48])|[13579][26])|([13579][26]|[02468][048]|0[0-9]|1[0-6])00)[-/]?02[-/]?29)$'

# Tabla
tb_consolidado_emp = 'consolidado_empresa'
tb_consolidado_per = 'consolidado'
tb_file_negativo_entel = 'entel_persona_file_negativo_productivo'
tb_rutero_uat = 'rutero_uat_contingencias'
tb_base_entel = 'tbtas_base_entel'
tb_household_princ = 'hh_principal_hist'
tb_household_staging = 'house_hold_resumen_jun21_staging'
tb_monedas_norm = 'monedas_norm'
tb_cons_contactabilidad = 'consolidado_contactabilidad_ref'
tb_personas_semanal = 'personas_semanal'
tb_empresas_semanal = 'empresas_semanal'
tb_vehiculos = 'tbtas_vehiculos'
tb_rubros = 'rubros'
p_exp_predictor_publico = 'p_exp_predictor_publico'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## TRANSFORMACIONES

In [27]:
df_cons = rubros(df_act_princ, df_act_sii, df_cons_per , df_rubro)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
dff = calculo_tablon_contingencia(df_cons_per, df_cons_emp, df_file_neg, df_base_entel, df_uf, df_hh, df_contactabilidad, df_pers_sem, df_sociedades, df_vehiculos, df_bancos, df_docs_trib, df_dir, df_act_princ, df_act_sii, df_rubro)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df_1 = variables_por_defecto(dff)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
df =  menores_edad_ko(df_1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## VER DATA FRAME

In [31]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+---+-----------+---------+---------+------------------------+------+-------------+--------------------+--------------------+------------------+--------------------+---------------+-------------+-------------------+-----+------------+---------------+---------------------+-----------+------+------+--------------------+-------------+-------------+---------------+------------+----------+---------------+--------------+-------------+-------------+------------+-------------------+---------------+------------+-----------+-------+---+---------------+-------------------+-------------+--------------+--------+-----------------+------------------+-----------------+--------------+--------------------+---------+---------+-----------+-------------+---------------+-------------------------+-------------------------+--------------------+--------------------+--------------------+---------------+---------+----------------

## ESCRIBIR DATOS

In [32]:
df.coalesce(1).write.mode('overwrite').csv('gs://dfa-dna-ws0011-la-prd-sandbox/contingencia_entel_semanal/contingencia_entel_semanal')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…